In [11]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import re

word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

[nltk_data] Downloading package punkt to /Users/sunhokim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sunhokim/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [61]:
con_df = pd.read_csv('concrete.csv', delimiter = ',')
con_df.set_index('word', inplace=True)

KeyError: 'afddsa'

In [65]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
def remove_stopwords(sen):
  sen_new = " ".join([i for i in sen if i not in stop_words])
  return sen_new
def summarize_sentences(sentences, n):
  clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
  clean_sentences = [s.lower() for s in clean_sentences]
  clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
  sentence_vectors = []
  for i in clean_sentences:
    if len(i) != 0:
      v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
      v = np.zeros((100,))
    sentence_vectors.append(v)

  sim_mat = np.zeros([len(sentences), len(sentences)])
  from sklearn.metrics.pairwise import cosine_similarity
  for i in range(len(sentences)):
    for j in range(len(sentences)):
      if i != j:
        sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
  import networkx as nx

  nx_graph = nx.from_numpy_array(sim_mat)
  scores = nx.pagerank(nx_graph)
  ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
  return [ranked_sentences[i][1] for i in range(n)]


from nltk.corpus import wordnet
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import simplejson
book = None
lemmatizer = WordNetLemmatizer() 
with open('out.book', 'rb') as f:
  book = simplejson.load(f)
fdist = FreqDist()
for chap in book['chapters']:
  for item in chap['items']:
    words = word_tokenize(item['content'])
    words = pd.Series(words).str.replace("[^a-zA-Z]", " ")
    words = [remove_stopwords(r.split()) for r in words]
    for word in words:
      word2 = lemmatizer.lemmatize(word.lower())
      fdist[word2] += 1
for chap in book['chapters']:
  if chap['id'] == '2c1d5e56-c074-11e9-8a54-48bf6beb4102':
    sens = summarize_sentences([item['content'] for item in chap['items']], 10)
    for sen in sens:
      keyword = None

      minScore = 9999999999999999999999
      words = word_tokenize(sen)
      words = pd.Series(words).str.replace("[^a-zA-Z]", " ")
      words = [remove_stopwords(r.split()) for r in words]
      for word in words:
        word2 = lemmatizer.lemmatize(word.lower())
        
        try:
          con = con_df.loc[word2]['conc']
        except KeyError:
          con = 5
        if con is not None:
          if fdist[word2] * 2 + con < minScore:
            minScore = fdist[word2] * 2 + con
            keyword = word
      print("keyword: " + keyword + " minFreq: " + str(minScore) + " sen: " + sen)

keyword: butterfly minFreq: 6.93 sen: "Well, perhaps you haven't found it so yet," said Alice, "but when you have to turn into a chrysalis—you will some day, you know—and then after that into a butterfly, I should think you'll feel it a little queer, won't you?"
keyword: adding minFreq: 7 sen: This was such a new idea to Alice, that she was quite silent for a minute or two, which gave the Pigeon the opportunity of adding, "You're looking for eggs, I know that well enough; and what does it matter to me whether you're a little girl or a serpent?"
keyword: girls minFreq: 18.85 sen: "I've seen a good many little girls in my time, but never one with such a neck as that!
keyword: immense minFreq: 4.21 sen: said Alice in a tone of delight, which changed into alarm in another moment, when she found that her shoulders were nowhere to be found: all she could see, when she looked down, was an immense length of neck, which seemed to rise like a stalk out of a sea of green leaves that lay far below